In [1]:
%cd /content
!cp -r /content/drive/MyDrive/ISSHACK/ . 

/content


In [6]:
!nvidia-smi

Sun Jan 23 07:07:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/edgecase-ai/Detectron-Using-Synthetic-Data

Cloning into 'Detectron-Using-Synthetic-Data'...
remote: Enumerating objects: 1388, done.
remote: Total 1388 (delta 0), reused 0 (delta 0), pack-reused 1388
Receiving objects: 100% (1388/1388), 10.12 MiB | 30.67 MiB/s, done.
Resolving deltas: 100% (826/826), done.


In [3]:
%cd Detectron-Using-Synthetic-Data/

/content/Detectron-Using-Synthetic-Data


In [4]:
!pip install -r requirements.txt
!make

python setup.py develop --user
Compiling detectron/utils/cython_bbox.pyx because it changed.
Compiling detectron/utils/cython_nms.pyx because it changed.
[1/2] Cythonizing detectron/utils/cython_bbox.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Detectron-Using-Synthetic-Data/detectron/utils/cython_bbox.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing detectron/utils/cython_nms.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Detectron-Using-Synthetic-Data/detectron/utils/cython_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating Detectron.egg-info
writing Detectron.egg-info/PKG-INFO
writing dependency_

In [5]:
!bash ./scripts/download_models.sh

899MB [00:07, 127MB/s]


In [ ]:
import sys
import argparse
import cv2

def extractImages(pathIn, pathOut,frame_stride_ms,start,end):
    count = start
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    while success and (count<=end):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_stride_ms))    # added this line 
        success,image = vidcap.read()
        print ('Read a new frame: ', success)
        if success:
          cv2.imwrite( pathOut + "frame%d.jpg" % count, image)     # save frame as JPEG file
        count = count + 1


In [62]:
!mkdir frame_all_train

In [63]:
extractImages('/content/ISSHACK/weapon_train.mp4','/content/Detectron-Using-Synthetic-Data/frame_all_train/',1000, 0,500)

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [66]:
!python tools/infer_simple_1.py \
    --cfg configs/gun/real.yaml \
    --output-dir /content/ \
    --image-ext jpg \
    --wts detectron/datasets/data/trained_weights/syntheticThenReal.pkl \
    /content/Detectron-Using-Synthetic-Data/frame_all_train

Found Detectron ops lib: /usr/local/lib/python3.7/dist-packages/torch/lib/libcaffe2_detectron_ops_gpu.so
[E init_intrinsics_check.cc:43] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature avx2 is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:43] CPU feature fma is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
WARNING cnn.py:  25: [====DEPRECATE WARNING====]: you are creating an object from CNNModelHelper class which will be deprecated soon. Please use ModelHelper object with brew module. For more information, please refer to caffe2.ai and python/brew.py, python/brew_test.py for more information.
INFO net.py:  60: Loading weights from: detectron/datasets/data/trained_we

In [69]:
!python /content/ISSHACK/examination_train.py /content/ISSHACK/ans_0.501_fix.txt /content/ISSHACK/weapon_train.txt

0.7899543378995434


In [60]:
with open('/content/ISSHACK/ans_0.501.txt','r') as f:
  lines = f.readlines()

new_dets = [int(line[-2]) for line in lines]
for z in range(1, len(lines)-1):
  if (int(lines[z][-2]) == 1) and  (int(lines[z-1][-2]) == 0)  and  (int(lines[z+1][-2]) == 0):
    new_dets[z] = 0


lines = []
for k,flag in enumerate(new_dets[:]):
  lines.append(str(k) + ' ' + str(k+1)+ ' ' + str(flag) + '\n')
with open('/content/ISSHACK/ans_'+ 'fix'+'.txt','w') as f:
  f.writelines(lines)



In [ ]:
while True:
  pass

In [ ]:
lines = []
for k,flag in enumerate(d_list[:-1]):
  lines.append(str(k) + ' ' + str(k+1)+ ' ' + str(flag) + '\n')

with open('/content/ISSHACK/ans.txt','w') as f:
  f.writelines(lines)

In [46]:
extractImages('/content/ISSHACK/weapon_train.mp4','/content/Detectron-Using-Synthetic-Data/frame_part_train/',1000, 100,300)

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame